In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm
pd.options.display.max_columns = None

In [2]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials ## para poner mis credenciales

# spotify
Aquí está la docu: https://spotipy.readthedocs.io/en/2.22.0/#

Configuro el cliente y le metó las keys para acceder a la API de Spotify y poder sacar datos de las canciones que tengo

In [35]:
id=os.getenv("id")
secret=os.getenv("secret")

client_credentials_manager = SpotifyClientCredentials(client_id=id,
                                                      client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Primero voy a sacar el URI de mis canciones. Para ello uso el método de spotipy.search, que permite buscar con una query el nombre del artista y la canción.

In [36]:
## esto es para obtener una uri --> me voy a sacar los uris para mi data frame
artist = 'los del rio'
track = 'la macarena'
q = 'artist:{} track: {}'.format(artist, track)
results = sp.search(q=q, limit=1, type='track')
print(results['tracks']['items'][0]['uri'])

spotify:track:3mbmmddF7fxUNFkXV91ecm


In [37]:
for index, row in tracks.iterrows():
    print(row['artist'],'---', row['name'])
    break

SZA --- Kill Bill


In [38]:
uris = []
for index, row in tracks.iterrows():
    q = 'artist:{} track: {}'.format(row['artist'], row['name'])
    res = sp.search(q=q, limit=1, type='track')
    try:
        uri = res['tracks']['items'][0]['uri']
        #print(res['tracks']['items'][0]['uri'])
        uris.append(uri)
    except:
        uris.append('')
    #break

Voy a comprobar que la lista donde me he appendeado todo tiene la misma longitud que mi dataframe, para poder añadir los URIs como una columna

In [39]:
uris[:3]

['spotify:track:3OHfY25tqY28d16oZczHc8',
 'spotify:track:0V3wPSX9ygBnCm8psDIegu',
 'spotify:track:5xMw6qCcpd2gBXPGTegC4W']

In [40]:
print(len(uris), tracks.shape)

450 (450, 9)


In [41]:
tracks['URI'] = uris
tracks.head()

,name,duration,playcount,listeners,url,artist,MBID,Streamable,tags,URI
0,Kill Bill,0,2946397,371889,https://www.last.fm/music/SZA/_/Kill+Bill,SZA,272989c8-5535-492d-a25c-9f58803e027f,0,NaN,spotify:track:3OHfY25tqY28d16oZczHc8
1,Anti-Hero,0,10077122,560069,https://www.last.fm/music/Taylor+Swift/_/Anti-...,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,0,NaN,spotify:track:0V3wPSX9ygBnCm8psDIegu
2,SOS,0,995490,307189,https://www.last.fm/music/SZA/_/SOS,SZA,272989c8-5535-492d-a25c-9f58803e027f,0,NaN,spotify:track:5xMw6qCcpd2gBXPGTegC4W
3,As It Was,0,20588530,909911,https://www.last.fm/music/Harry+Styles/_/As+It...,Harry Styles,,0,NaN,spotify:track:4LRPiXqCikLlN15c3yImP7
4,Shirt,0,3121081,359910,https://www.last.fm/music/SZA/_/Shirt,SZA,272989c8-5535-492d-a25c-9f58803e027f,0,NaN,spotify:track:34ZAzO78a5DAVNrYIGWcPm


Ahora con el método `'audio_features'` de spotipy quiero sacar las propiedades que tienen las canciones. Para ello, tengo que aportar:
- `URI`

Y el resultado de cada una de mis consultas, lo almacenaré en una lista de listas que luego appendearé a mi dataframe.

In [42]:
sp.audio_features('spotify:track:3OHfY25tqY28d16oZczHc8')

[{'danceability': 0.644,
  'energy': 0.728,
  'key': 8,
  'loudness': -5.75,
  'mode': 1,
  'speechiness': 0.0351,
  'acousticness': 0.0543,
  'instrumentalness': 0.169,
  'liveness': 0.161,
  'valence': 0.43,
  'tempo': 88.993,
  'type': 'audio_features',
  'id': '3OHfY25tqY28d16oZczHc8',
  'uri': 'spotify:track:3OHfY25tqY28d16oZczHc8',
  'track_href': 'https://api.spotify.com/v1/tracks/3OHfY25tqY28d16oZczHc8',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3OHfY25tqY28d16oZczHc8',
  'duration_ms': 153947,
  'time_signature': 4}]

In [43]:
type(sp.audio_features('spotify:track:3OHfY25tqY28d16oZczHc8'))

list

In [44]:
datos_uris = []
for index, row in tracks.iterrows():
    uri = sp.audio_features(row['URI'])
    datos_uris.append(uri)

Y tengo mis datos sobre las canciones, pero tengo que limpiar un poco para poder hacer el append.

In [51]:
type(pd.DataFrame(datos_uris).iloc[0,0]) ## puedo sacar las claves para nombrar a mis columnas

dict

In [53]:
columnas = list(pd.DataFrame(datos_uris).iloc[0,0].keys())
columnas[:5]

['danceability', 'energy', 'key', 'loudness', 'mode']

In [56]:
df_uris = pd.DataFrame(datos_uris)
df_uris[columnas] = df_uris[0].apply(pd.Series)
df_uris.head()

C:\Users\annas\AppData\Local\Temp\ipykernel_14644\1750135214.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_uris[columnas] = df_uris[0].apply(pd.Series)
C:\Users\annas\AppData\Local\Temp\ipykernel_14644\1750135214.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_uris[columnas] = df_uris[0].apply(pd.Series)
C:\Users\annas\AppData\Local\Temp\ipykernel_14644\1750135214.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_uris[columnas] = df_uris[0].apply(pd.Series)
C:\Users\annas\AppData\Local\Temp\ipykernel_14644\1750135214.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future

,0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,"{'danceability': 0.644, 'energy': 0.728, 'key'...",0.644,0.728,8.0,-5.750,1.0,0.0351,0.0543,0.169000,0.1610,0.430,88.993,audio_features,3OHfY25tqY28d16oZczHc8,spotify:track:3OHfY25tqY28d16oZczHc8,https://api.spotify.com/v1/tracks/3OHfY25tqY28...,https://api.spotify.com/v1/audio-analysis/3OHf...,153947.0,4.0
1,"{'danceability': 0.637, 'energy': 0.643, 'key'...",0.637,0.643,4.0,-6.571,1.0,0.0519,0.1300,0.000002,0.1420,0.533,97.008,audio_features,0V3wPSX9ygBnCm8psDIegu,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBn...,https://api.spotify.com/v1/audio-analysis/0V3w...,200690.0,4.0
2,"{'danceability': 0.507, 'energy': 0.657, 'key'...",0.507,0.657,7.0,-7.356,0.0,0.2330,0.6690,0.000007,0.0947,0.506,119.159,audio_features,5xMw6qCcpd2gBXPGTegC4W,spotify:track:5xMw6qCcpd2gBXPGTegC4W,https://api.spotify.com/v1/tracks/5xMw6qCcpd2g...,https://api.spotify.com/v1/audio-analysis/5xMw...,117773.0,1.0
3,"{'danceability': 0.52, 'energy': 0.731, 'key':...",0.520,0.731,6.0,-5.338,0.0,0.0557,0.3420,0.001010,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303.0,4.0
4,"{'danceability': 0.824, 'energy': 0.453, 'key'...",0.824,0.453,3.0,-9.604,0.0,0.0968,0.1460,0.027300,0.0896,0.552,119.959,audio_features,34ZAzO78a5DAVNrYIGWcPm,spotify:track:34ZAzO78a5DAVNrYIGWcPm,https://api.spotify.com/v1/tracks/34ZAzO78a5DA...,https://api.spotify.com/v1/audio-analysis/34ZA...,181831.0,4.0


Lo junto ahora con mi dataframe inicial y limpio las columnas que no necesite.

In [58]:
tracks = tracks.merge(df_uris, left_on = 'URI', right_on='uri', indicator=True)
tracks['_merge'].value_counts() ## compruebo que el cruce se haga con éxito

both          447
left_only       0
right_only      0
Name: _merge, dtype: int64

In [62]:
tracks.drop([0, 'uri', '_merge'], axis = 1, inplace=True)
tracks.head()

,name,duration,playcount,listeners,url,artist,MBID,Streamable,tags,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Kill Bill,0,2946397,371889,https://www.last.fm/music/SZA/_/Kill+Bill,SZA,272989c8-5535-492d-a25c-9f58803e027f,0,NaN,spotify:track:3OHfY25tqY28d16oZczHc8,0.644,0.728,8.0,-5.750,1.0,0.0351,0.0543,0.169000,0.1610,0.430,88.993,audio_features,3OHfY25tqY28d16oZczHc8,https://api.spotify.com/v1/tracks/3OHfY25tqY28...,https://api.spotify.com/v1/audio-analysis/3OHf...,153947.0,4.0
1,Anti-Hero,0,10077122,560069,https://www.last.fm/music/Taylor+Swift/_/Anti-...,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,0,NaN,spotify:track:0V3wPSX9ygBnCm8psDIegu,0.637,0.643,4.0,-6.571,1.0,0.0519,0.1300,0.000002,0.1420,0.533,97.008,audio_features,0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBn...,https://api.spotify.com/v1/audio-analysis/0V3w...,200690.0,4.0
2,SOS,0,995490,307189,https://www.last.fm/music/SZA/_/SOS,SZA,272989c8-5535-492d-a25c-9f58803e027f,0,NaN,spotify:track:5xMw6qCcpd2gBXPGTegC4W,0.507,0.657,7.0,-7.356,0.0,0.2330,0.6690,0.000007,0.0947,0.506,119.159,audio_features,5xMw6qCcpd2gBXPGTegC4W,https://api.spotify.com/v1/tracks/5xMw6qCcpd2g...,https://api.spotify.com/v1/audio-analysis/5xMw...,117773.0,1.0
3,As It Was,0,20588530,909911,https://www.last.fm/music/Harry+Styles/_/As+It...,Harry Styles,,0,NaN,spotify:track:4LRPiXqCikLlN15c3yImP7,0.520,0.731,6.0,-5.338,0.0,0.0557,0.3420,0.001010,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303.0,4.0
4,Shirt,0,3121081,359910,https://www.last.fm/music/SZA/_/Shirt,SZA,272989c8-5535-492d-a25c-9f58803e027f,0,NaN,spotify:track:34ZAzO78a5DAVNrYIGWcPm,0.824,0.453,3.0,-9.604,0.0,0.0968,0.1460,0.027300,0.0896,0.552,119.959,audio_features,34ZAzO78a5DAVNrYIGWcPm,https://api.spotify.com/v1/tracks/34ZAzO78a5DA...,https://api.spotify.com/v1/audio-analysis/34ZA...,181831.0,4.0
